<a href="https://colab.research.google.com/github/porcelainruler/Kaggle/blob/main/HousePricePrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Library Import
import os

In [5]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [11]:
# /content/gdrive/My Drive/Kaggle is the path where kaggle.json is present in the Google Drive
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle/HousePredictionKaggle"

#changing the working directory
%cd /content/gdrive/My Drive/Kaggle/HousePredictionKaggle

#Check the present working directory using pwd command

/content/gdrive/My Drive/Kaggle/HousePredictionKaggle
